In [ ]:
import os
import numpy as np
import cv2
from skimage.io import imread, imsave,  imshow
from skimage.feature import hog
from skimage import exposure

## Function for content and path extraction


In [ ]:
def dir_content_path(name):
# path for current working directory and directories in it
    
    path = os.getcwd()
    dirs = next(os.walk(path))[1]

    # getting 'videos' folder index and path and its content
    dir_Index = (dirs.index(name))
    dir_Path = os.path.join(os.getcwd(),dirs[dir_Index])
    dir_content = next(os.walk(dir_Path))[2]
    directories = next(os.walk(dir_Path))[1]
    return dir_content, dir_Path, directories

## Function for video2frame conversion

In [ ]:
def v2f (video_dir_content,video_dir_path,frames_dir_path):

    for video_count in video_dir_content:
            # slpiting video name for making classes and labels
            frames_dir_name = video_count.split('.')[0] 
            frames_new_dir_path = os.path.join(frames_dir_path,frames_dir_name)

            # creating individual class folder
            os.mkdir(frames_new_dir_path)
            print('folder for frames of video %s is created' %frames_dir_name)

            video2frame_path = os.path.join(video_dir_path, video_count)

            # video to frames conversion
            cap = cv2.VideoCapture(video2frame_path)
            frames,img = cap.read()
            count = 0
            while frames:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                cv2.imwrite(frames_new_dir_path+'/frame '+ str(count)+'.jpg', img) # save frame as JPEG file      
                frames,img = cap.read()
                count += 1
    print('All Videos are converted to frames')

## Function for HOG algorithm

In [ ]:
def HOG(frames_dir_directories, frames_dir_path, HOG_dir_path ): 

    for frames_dir_count in frames_dir_directories:

        X = []
        y = []

        # joining HOG folder path and individual class name and create folder
        HOG_dir = os.path.join(HOG_dir_path,frames_dir_count)

        os.mkdir(HOG_dir)
        print('folder for HOG pictures %s is created' %HOG_dir)

        frames_dir = os.path.join(frames_dir_path,frames_dir_count)
        frames_index =next(os.walk(frames_dir))[2]


        # loop for accessing individual frames inside respective directory
        for frames_count in frames_index:
            print(frames_dir_count)
            label = frames_dir_count.split(' ')[0] # get indiviual class as label for example walking,boxing
            frames = os.path.join(frames_dir,frames_count) # individual frame path
            HOG_frames = os.path.join(HOG_dir,frames_count) # indiviual HOG path
            if (label=='boxing'):

                targets = 1
            elif ( label=='handclapping'):
                targets = 2
            elif ( label=='handwaving'):
                targets = 3
            elif ( label=='jogging'):
                targets = 4 
            elif ( label=='running'):
                targets = 5 
            elif ( label=='walking'):
                targets = 6 
            print('label value:', label)
            print('target value:', targets)

            img = imread(frames) #image reading
            # applying HOG algorthm using skimage buitin features
            fd, hog_image = hog(img, orientations=9, pixels_per_cell=(8, 8),
                        cells_per_block=(2, 2), visualize=True)
            hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
            imsave(HOG_frames,hog_image_rescaled,as_grey=True, check_contrast=False) # saving HOG covnerted image to indiviudal folder
            # appending features of individual frames and respective label to X and y Lists
            X.append(fd)
            y.append(targets)
        t=np.array(X,dtype=float)
        u=np.array(y,dtype=float)
        with open(frames_dir_count+'_features.npy','wb') as f:
            np.save(f,t)

        with open(frames_dir_count+'_targets.npy','wb') as f:
            np.save(f,u)




# Main code

In [ ]:
video_dir_content, video_dir_path, video_dir_directories = dir_content_path('videos')
frames_dir_content, frames_dir_path, frames_dir_directories = dir_content_path('frames')
v2f(video_dir_content,video_dir_path,frames_dir_path)


In [ ]:
frames_dir_content, frames_dir_path, frames_dir_directories = dir_content_path('frames')
HOG_dir_content, HOG_dir_path, HOG_dir_directories = dir_content_path('HOG')
HOG(frames_dir_directories, frames_dir_path, HOG_dir_path )